In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import preprocessing

import statsmodels.api as sm
from statsmodels.tsa.stattools import coint

from scipy import stats

from quantopian.pipeline.data import morningstar
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume
from quantopian.pipeline.filters import  StaticAssets
from quantopian.pipeline.filters.morningstar import Q500US, Q1500US, Q3000US
from quantopian.pipeline.data.factset import EquityMetadata
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

In [ ]:
security_type = EquityMetadata.security_type.latest
close = USEquityPricing.close.latest
dollar_volume = AverageDollarVolume(window_length=30)

high_dollar_volume = dollar_volume.percentile_between(90, 100)
is_ETF = security_type.startswith('ETF')

# Create a pipline with each of the factor outputs as columns
pipe = Pipeline(
            columns = {
            'close' : close,
            'dollar_vol' : dollar_volume,
            })
pipe.set_screen(is_ETF)

In [ ]:
etf = run_pipeline(pipe, '2019-03-01', '2019-03-01')
etf = etf.reset_index(level=0, drop=True)
etf.head()

In [ ]:
counts, bins, bars = plt.hist(etf['dollar_vol'], bins = 100)
plt.show()

#print("{0} ETFs have daily trading volumes in the bottom {1}th percentile, below {2}."
      #.format(int(counts[0]), int(counts[0]/len(etf)*100), int(bins[1])))

In [ ]:
percentile = etf['dollar_vol'].quantile(0.95) #dollar vol of top 90th percentile
liquid_etf = etf[etf['dollar_vol']> percentile] #filter for top 10% most liquid ETF
liquid_etf = liquid_etf.sort_values(by='dollar_vol',ascending=False) #sort ETF by dollar volume
liquid_etf.head()

In [ ]:
def find_cointegrated_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.01:
                pairs.append((keys[i], keys[j]))
    return score_matrix, pvalue_matrix, pairs

In [ ]:
etf_universe = []
for etf in liquid_etf.index:
    etf_universe.append(etf.symbol)

In [ ]:
universe = StaticAssets(symbols(etf_universe))

In [ ]:
filtered_pipe = Pipeline(
            columns = {
            'close' : close},
            screen = universe)

liquid_pipe = run_pipeline(filtered_pipe, '2018-08-01', '2019-08-01')

In [ ]:
prices = liquid_pipe.close.unstack()
prices.head()

In [ ]:
scores, pvalues, pairs = find_cointegrated_pairs(prices)

In [ ]:
import seaborn
seaborn.heatmap(pvalues, xticklabels=etf_universe, yticklabels=etf_universe, cmap='RdYlGn_r' 
                , mask = (pvalues >= 0.05)
                )
pairs

In [ ]:
for pair in pairs:
    print("(symbol('{0}'), symbol('{1}')),".format(pair[0].symbol, pair[1].symbol))

In [ ]:
for etf in np.unique(pairs[0:50]):
    print("'{0}',".format(etf.symbol))

In [ ]:
stock_pairs = [ (symbols('SPY'), symbols('IVV')),
                (symbols('SPY'), symbols('IWB')),
                (symbols('SPY'), symbols('FEZ')),
                (symbols('SPY'), symbols('VOO'))]

stocks = symbols(['SPY','FEZ','VOO','IJH','MDY','VEA'])
num_pairs = len(stock_pairs)

# strategy specific variables
lookback = 20 # used for regression
z_window = 20 # used for zscore calculation, must be <= lookback
target_weights = pd.Series(index=stocks, data=0.25)

In [ ]:
for pair in pairs[0:50]:
    print("(symbol('{0}'), symbol('{1}')),".format(pair[0].symbol, pair[1].symbol))

In [ ]:
def get_weight(context,data,pair,dev_cutoff,dev_ceil=3):
    price_diff=context.price_hist[pair[0]].sub(context.price_hist[pair[1]])
    mean_diff=price_diff.mean()
    sd_diff=price_diff.std()
    price1 = data.current(pair[0],"price")
    price2 = data.current(pair[1],"price")
    cur_diff = price1 - price2
    cur_deviation = abs(cur_diff - mean_diff)/sd_diff
    mprice=max(price1,price2)
    weight=0
    if (dev_ceil !=3):
        print "Hm %s,Cm %s, sd %s,cd %s,mp %s" % (mean_diff,cur_diff,sd_diff,cur_deviation,mprice)
    #Discard high volatile stock
    if (sd_diff/mprice > .1):
        return 0
    if (cur_deviation > dev_cutoff and cur_deviation < dev_ceil):
        if (cur_diff > mean_diff):
            weight=(-1.0*cur_deviation)
        else:
            weight=(1.0*cur_deviation)
    return weight   